<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/auto-keras/california_housing/xgb_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
%pip show xgboost

Name: xgboost
Version: 0.90
Summary: XGBoost Python Package
Home-page: https://github.com/dmlc/xgboost
Author: None
Author-email: None
License: Apache-2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: scipy, numpy
Required-by: 


In [28]:
%cd /content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
%ls

/content/drive/My Drive/機械学習練習/AutoML/auto-keras/california_housing
auto_keras_outputs/        datasets/         xgboost_reg.ipynb
autokeras_regressor.ipynb  fetch_data.ipynb


In [39]:
import os, pprint

import pandas as pd
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score

from xgboost import XGBRegressor

## データ読み込み

In [30]:
F_COLS = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
TARGET_COL = 'MedHouseVal'

data_path = 'datasets/origin'
train_csv = 'train.csv'
val_csv = 'val.csv'
test_csv = 'test.csv'

train_df= pd.read_csv(os.path.join(data_path, train_csv), header=0)
val_df = pd.read_csv(os.path.join(data_path, val_csv), header=0)
test_df = pd.read_csv(os.path.join(data_path, test_csv), header=0)

print('train shape:{}'.format(train_df.shape))
print('validation shape:{}'.format(val_df.shape))
print('test shape:{}'.format(test_df.shape))

display(train_df.head())

train shape:(13209, 9)
validation shape:(3303, 9)
test shape:(4128, 9)


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,4.4063,15.0,6.104430,1.003165,1237.0,3.914557,34.00,-117.50,1.285
1,3.7222,35.0,5.539326,1.030899,1152.0,3.235955,33.87,-117.96,1.875
2,1.1094,49.0,3.773585,1.144654,1448.0,4.553459,37.95,-121.28,0.525
3,3.3239,9.0,5.610955,1.092697,1691.0,2.375000,36.82,-119.85,0.913
4,5.9629,17.0,6.867133,1.097902,808.0,2.825175,38.58,-121.81,2.860


## XGBoos回帰モデルを構築

In [36]:
SEED = 7

reg = XGBRegressor(n_estimators=1000,
                   seed=SEED)
reg

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=7, silent=None,
             subsample=1, verbosity=1)

In [37]:
%time
reg.fit(X=train_df.drop(columns=TARGET_COL).values,
        y=train_df[TARGET_COL].values,
        eval_set=[(val_df.drop(columns=TARGET_COL).values, val_df[TARGET_COL].values)],
        early_stopping_rounds=3)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs
[06:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:1.82163
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:1.67765
[2]	validation_0-rmse:1.54937
[3]	validation_0-rmse:1.43711
[4]	validation_0-rmse:1.33808
[5]	validation_0-rmse:1.25086
[6]	validation_0-rmse:1.17508
[7]	validation_0-rmse:1.10842
[8]	validation_0-rmse:1.05154
[9]	validation_0-rmse:0.999667
[10]	validation_0-rmse:0.955497
[11]	validation_0-rmse:0.918296
[12]	validation_0-rmse:0.88404
[13]	validation_0-rmse:0.851828
[14]	validation_0-rmse:0.825755
[15]	validation_0-rmse:0.80357
[16]	validation_0-rmse:0.784042
[17]	validation_0-rmse:0.767869
[18]	validation_0-rmse:0.751435
[19]	validation_0-rmse:0.738701
[20]	validation_0-rmse:0.724905
[21]	validation_0-rmse:0.714596
[22]	validation_0-rmse:0.705812
[23]	validation_0-rmse:0.69599

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=7, silent=None,
             subsample=1, verbosity=1)

## モデルの評価

### 評価指標の定義

In [40]:
SK_METRICS=[r2_score, explained_variance_score, mean_squared_error]

def _calc_score(y_true, y_pred, sklearn_metric):
  '''sklearnの評価指標のスコアを計算'''
  score = sklearn_metric(y_true=y_true, y_pred=y_pred)
  return score


def evaluete_sk_metrics(y_true, y_pred, sklearn_metrics):
  '''sklearnの各種、評価関数を使用'''
  result = {}
  for sk_metric in sklearn_metrics:
    result[sk_metric.__name__] = _calc_score(y_true=y_true, y_pred=y_pred, sklearn_metric=sk_metric)
  return result

### 推論結果の取得

In [44]:
def predict_multiple_times(model, inputs, input_names):
  '''何度か推論'''
  pred = {}
  for input, name in zip(inputs, input_names):
    pred[name] = model.predict(input)
  return pred

In [45]:
%time
reg_inputs = [train_df[F_COLS].values, val_df[F_COLS].values, test_df[F_COLS].values]
reg_input_names = [train_csv, val_csv, test_csv]

reg_preds = predict_multiple_times(model=reg, inputs=reg_inputs, input_names=reg_input_names)
reg_preds

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


{'test.csv': array([2.165414 , 2.4882603, 2.9066982, ..., 1.5247506, 3.3332872,
        4.407699 ], dtype=float32),
 'train.csv': array([1.4310822, 1.750393 , 0.7285534, ..., 1.6013324, 1.3429168,
        2.0459855], dtype=float32),
 'val.csv': array([0.9650786, 1.0881988, 1.4906633, ..., 0.9098087, 0.6162703,
        0.9352584], dtype=float32)}

### 評価の実行

In [46]:
eval_scores = {}

for df, key in zip([train_df, val_df, test_df], [train_csv, val_csv, test_csv]):
  eval_scores[key] = evaluete_sk_metrics(y_true=df[TARGET_COL].values,
                                              y_pred=reg_preds[key], sklearn_metrics=SK_METRICS)

pprint.pprint(eval_scores)

{'test.csv': {'explained_variance_score': 0.7959080889556576,
              'mean_squared_error': 0.2747132700861737,
              'r2_score': 0.795754720018762},
 'train.csv': {'explained_variance_score': 0.8314111332558983,
               'mean_squared_error': 0.22121462647575077,
               'r2_score': 0.8314111300117255},
 'val.csv': {'explained_variance_score': 0.8046547107411665,
             'mean_squared_error': 0.27136591506967356,
             'r2_score': 0.8045228381407727}}
